In [1]:
!pip install tensorflow pretty_midi


     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB 660.6 kB/s eta 0:00:09
     ---------------------------------------- 0.1/5.6 MB 656.4 kB/s eta 0:00:09
     - -------------------------------------- 0.1/5.6 MB 950.9 kB/s eta 0:00:06
     - -------------------------------------- 0.2/5.6 MB 1.1 MB/s eta 0:00:06
     - -------------------------------------- 0.2/5.6 MB 1.1 MB/s eta 0:00:06
     --- ------------------------------------ 0.5/5.6 MB 1.8 MB/s eta 0:00:03
     --- ------------------------------------ 0.5/5.6 MB 1.5 MB/s eta 0:00:04
     --- ------------------------------------ 0.5/5.6 MB 1.4 MB/s eta 0:00:04
     ----- ---------------------------------- 0.8/5.6 MB 1.9 MB/s eta 0:00:03
     ------ --------------------------------- 1.0/5.6 MB 1.9 MB/s eta 0:00:03
     --------- ------------------------------ 1.4/5.6 MB 2.4 MB/s eta 0:00:02
     ---------- ----------------------------- 1.4/5.6 MB 2.6 MB/s

In [2]:
import os
import pretty_midi

def ler_arquivos_midi(caminho_pasta):
    arquivos_midi = []
    for arquivo in os.listdir(caminho_pasta):
        if arquivo.endswith('.mid') or arquivo.endswith('.midi'):
            caminho = os.path.join(caminho_pasta, arquivo)
            midi = pretty_midi.PrettyMIDI(caminho)
            arquivos_midi.append(midi)
    return arquivos_midi

# Caminho para a pasta com arquivos MIDI
caminho_pasta_midi = 'midi'
musicas_midi = ler_arquivos_midi(caminho_pasta_midi)


C:\Users\ealbvit\AppData\Local\anaconda3\envs\redes_neurais\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [4]:
def extrair_notas_e_duracoes(midi_file):
    notas_duracoes = []
    for instrument in midi_file.instruments:
        if not instrument.is_drum:
            for nota in instrument.notes:
                inicio = nota.start
                fim = nota.end
                duracao = fim - inicio
                nota_duracao = (nota.pitch, duracao)
                notas_duracoes.append(nota_duracao)
    return notas_duracoes

dados_midi = [extrair_notas_e_duracoes(midi) for midi in musicas_midi]


In [9]:
tamanho_sequencia = 20  # Número de pares nota-duração na sequência de entrada
entradas, saidas = [], []

for musica in dados_midi:
    for i in range(0, len(musica) - tamanho_sequencia):
        entrada = musica[i:i + tamanho_sequencia]
        saida = musica[i + tamanho_sequencia]
        entradas.append(entrada)
        saidas.append(saida)


In [6]:
#1. Mapeamento de Notas e Durações para Índices

notas = set()
duracoes = set()

for musica in dados_midi:
    for nota, duracao in musica:
        notas.add(nota)
        duracoes.add(duracao)

# Mapeamento de notas e durações para índices
nota_para_indice = {nota: i for i, nota in enumerate(sorted(notas))}
duracao_para_indice = {duracao: i for i, duracao in enumerate(sorted(duracoes))}


In [15]:
#One hot encoding 

import numpy as np

entradas = []
saidas = []

entradas = []
saidas = []

for musica in dados_midi:
    for i in range(0, len(musica) - tamanho_sequencia):
        entrada_seq = musica[i:i + tamanho_sequencia]
        saida_seq = musica[i + tamanho_sequencia]

        entrada = [(nota_para_indice[n], duracao_para_indice[d]) for n, d in entrada_seq]
        saida = (nota_para_indice[saida_seq[0]], duracao_para_indice[saida_seq[1]])

        entradas.append(entrada)
        saidas.append(saida)

from scipy.sparse import lil_matrix
from tqdm import tqdm

# Criar matrizes esparsas para entradas e saídas
entradas_one_hot = lil_matrix((len(entradas), tamanho_sequencia * (num_notas + num_duracoes)), dtype='float32')
saidas_one_hot = lil_matrix((len(saidas), num_notas + num_duracoes), dtype='float32')

# Usando tqdm para mostrar o progresso
for i in tqdm(range(len(entradas)), desc="Processando entradas"):
    seq = entradas[i]
    for j, (nota_indice, duracao_indice) in enumerate(seq):
        entradas_one_hot[i, j * (num_notas + num_duracoes) + nota_indice] = 1
        entradas_one_hot[i, j * (num_notas + num_duracoes) + num_notas + duracao_indice] = 1

for i in tqdm(range(len(saidas)), desc="Processando saídas"):
    nota_indice, duracao_indice = saidas[i]
    saidas_one_hot[i, nota_indice] = 1
    saidas_one_hot[i, num_notas + duracao_indice] = 1

    from scipy.sparse import csr_matrix

# Suponha que 'entradas_one_hot' e 'saidas_one_hot' são as suas matrizes one-hot
entradas_csr = csr_matrix(entradas_one_hot)
saidas_csr = csr_matrix(saidas_one_hot)


Processando saídas: 100%|██████████████████████████████████████████████████| 374362/374362 [00:03<00:00, 102120.66it/s]


In [17]:
# Definindo o Codificador (Encoder)

from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.models import Model

# Definindo o tamanho da sequência de entrada e o número de features
tamanho_sequencia = entradas_csr.shape[1]
num_features = entradas_csr.shape[1]

# Camada de entrada
entrada = Input(shape=(tamanho_sequencia, num_features))

# Codificador
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(entrada)

# Nós apenas precisamos dos estados, não das saídas
encoder_states = [state_h, state_c]


In [18]:
#Definindo o Decodificador (Decoder)

# Decodificador
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(RepeatVector(tamanho_sequencia)(encoder_outputs), initial_state=encoder_states)


In [28]:
class SparseDataGenerator(Sequence):
    def __init__(self, entradas_csr, saidas_csr, batch_size, tamanho_sequencia, num_features):
        self.entradas_csr = entradas_csr
        self.saidas_csr = saidas_csr
        self.batch_size = batch_size
        self.tamanho_sequencia = tamanho_sequencia
        self.num_features = num_features

    def __len__(self):
        return math.ceil(self.entradas_csr.shape[0] / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.entradas_csr[idx * self.batch_size:(idx + 1) * self.batch_size].todense()
        batch_y = self.saidas_csr[idx * self.batch_size:(idx + 1) * self.batch_size].todense()

        # Certifique-se de que o número total de elementos em cada lote é divisível por (tamanho_sequencia * num_features)
        if batch_x.size % (self.tamanho_sequencia * self.num_features) == 0:
            batch_x_reshaped = np.reshape(batch_x, (-1, self.tamanho_sequencia, self.num_features))
        else:
            raise ValueError("O tamanho dos dados não é compatível com o tamanho da sequência e o número de features.")

        if batch_y.size % (self.tamanho_sequencia * self.num_features) == 0:
            batch_y_reshaped = np.reshape(batch_y, (-1, self.tamanho_sequencia, self.num_features))
        else:
            raise ValueError("O tamanho dos dados não é compatível com o tamanho da sequência e o número de features.")

        return batch_x_reshaped, batch_y_reshaped

# Use o gerador para treinar o modelo
train_generator = SparseDataGenerator(entradas_csr, saidas_csr, batch_size=32, tamanho_sequencia=tamanho_sequencia, num_features=num_features)
model.fit(train_generator, epochs=50)


ValueError: O tamanho dos dados não é compatível com o tamanho da sequência e o número de features.

In [22]:
model.fit(entradas_csr, saidas_csr, batch_size=64, epochs=50)

Epoch 1/50


ValueError: in user code:

    File "C:\Users\ealbvit\AppData\Local\anaconda3\envs\redes_neurais\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\ealbvit\AppData\Local\anaconda3\envs\redes_neurais\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ealbvit\AppData\Local\anaconda3\envs\redes_neurais\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\ealbvit\AppData\Local\anaconda3\envs\redes_neurais\lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\ealbvit\AppData\Local\anaconda3\envs\redes_neurais\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(f

MemoryError: Unable to allocate 167. GiB for an array with shape (374362, 119620) and data type float32

In [30]:
!pip install music21

from music21 import converter
import os

# Carregar o arquivo MIDI
midi_path = './midi/Metallica_-_No_Remorse.mid'
midi = converter.parse(midi_path)

# Analisar a estrutura do MIDI
for i, track in enumerate(midi.parts):
    print(f"Track {i}: {track}")
    for element in track:
        print(f" - {element}")


     ---------------------------------------- 0.0/22.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/22.8 MB ? eta -:--:--
     --------------------------------------- 0.0/22.8 MB 487.6 kB/s eta 0:00:47
     --------------------------------------- 0.1/22.8 MB 744.7 kB/s eta 0:00:31
     ---------------------------------------- 0.3/22.8 MB 1.6 MB/s eta 0:00:15
      --------------------------------------- 0.5/22.8 MB 2.4 MB/s eta 0:00:10
     - -------------------------------------- 0.6/22.8 MB 2.5 MB/s eta 0:00:10
     - -------------------------------------- 0.9/22.8 MB 2.9 MB/s eta 0:00:08
     -- ------------------------------------- 1.2/22.8 MB 3.5 MB/s eta 0:00:07
     -- ------------------------------------- 1.6/22.8 MB 4.1 MB/s eta 0:00:06
     --- ------------------------------------ 2.1/22.8 MB 4.6 MB/s eta 0:00:05
     ---- ----------------------------------- 2.5/22.8 MB 4.9 MB/s eta 0:00:05
     ----- ---------------------------------- 2.9/22.8 MB 5.4 MB

Track 0: <music21.stream.Part 0x184348e6b20>
 - <music21.stream.Measure 1 offset=0.0>
 - <music21.stream.Measure 2 offset=4.0>
 - <music21.stream.Measure 3 offset=8.0>
 - <music21.stream.Measure 4 offset=12.0>
 - <music21.stream.Measure 5 offset=16.0>
 - <music21.stream.Measure 6 offset=20.0>
 - <music21.stream.Measure 7 offset=24.0>
 - <music21.stream.Measure 8 offset=28.0>
 - <music21.stream.Measure 9 offset=32.0>
 - <music21.stream.Measure 10 offset=36.0>
 - <music21.stream.Measure 11 offset=40.0>
 - <music21.stream.Measure 12 offset=44.0>
 - <music21.stream.Measure 13 offset=48.0>
 - <music21.stream.Measure 14 offset=52.0>
 - <music21.stream.Measure 15 offset=56.0>
 - <music21.stream.Measure 16 offset=60.0>
 - <music21.stream.Measure 17 offset=64.0>
 - <music21.stream.Measure 18 offset=68.0>
 - <music21.stream.Measure 19 offset=72.0>
 - <music21.stream.Measure 20 offset=76.0>
 - <music21.stream.Measure 21 offset=80.0>
 - <music21.stream.Measure 22 offset=84.0>
 - <music21.stream.Me